# Get focal length/image width

In [6]:
from PIL import Image
import exifread

# Load the image
image_path = "kb60.jpg"  # Replace with your image file path
image = Image.open(image_path)

# Get image width
image_width = image.width

# Open the image file for reading EXIF data
with open(image_path, 'rb') as img_file:
    tags = exifread.process_file(img_file)
    focal_length_raw = tags.get('EXIF FocalLength', 'Not available')
    focal_length_35mm_raw = tags.get('EXIF FocalLengthIn35mmFilm', 'Not available')
    
    # Convert focal length to float
    if focal_length_raw != 'Not available':
        focal_length = float(focal_length_raw.values[0])
    else:
        focal_length = 'Not available'
    
    # Convert 35mm focal length to int
    if focal_length_35mm_raw != 'Not available':
        focal_length_35mm = int(focal_length_35mm_raw.values[0])
    else:
        focal_length_35mm = 'Not available'

print(f"Image Width: {image_width}")
print(f"Focal Length: {focal_length} mm")
print(f"Focal Length (35mm equivalent): {focal_length_35mm} mm")

Image Width: 5712
Focal Length: 5.960000038146973 mm
Focal Length (35mm equivalent): 26 mm


In [8]:
import math

def estimate_sensor_dimensions(focal_length_mm, focal_length_35mm, aspect_ratio_width=4, aspect_ratio_height=3):
    """
    Estimate sensor width and height (mm) using crop factor and aspect ratio.
    
    Args:
        focal_length_mm: Actual focal length of the camera (mm)
        focal_length_35mm: 35mm equivalent focal length (mm)
        aspect_ratio_width: Width component of aspect ratio (default 3)
        aspect_ratio_height: Height component of aspect ratio (default 2)
        
    Returns:
        (sensor_width_mm, sensor_height_mm)
    """
    if not focal_length_mm or not focal_length_35mm:
        return None, None

    # Compute crop factor
    crop_factor = focal_length_35mm / focal_length_mm

    # Sensor width based on full-frame width (36 mm)
    sensor_width = 36.0 / crop_factor

    # Compute sensor height using aspect ratio
    sensor_height = sensor_width * (aspect_ratio_height / aspect_ratio_width)

    return sensor_width, sensor_height

# Ask user for input
aspect_ratio_w = 4
aspect_ratio_h = 3
# Estimate sensor dimensions
sensor_width_mm, sensor_height_mm = estimate_sensor_dimensions(focal_length, focal_length_35mm, aspect_ratio_w, aspect_ratio_h)

print(f"Estimated sensor width: {sensor_width_mm:.2f} mm")
print(f"Estimated sensor height: {sensor_height_mm:.2f} mm")


Estimated sensor width: 8.25 mm
Estimated sensor height: 6.19 mm


In [12]:
import math

def estimate_sensor_dimensions(focal_length_mm, focal_length_35mm, aspect_ratio_width=4, aspect_ratio_height=3):
    """
    Estimate sensor width, height, and diagonal (mm) from actual focal length,
    35mm equivalent focal length, and sensor aspect ratio.

    Args:
        focal_length_mm: Actual focal length of the camera (mm)
        focal_length_35mm: 35mm equivalent focal length (mm)
        aspect_ratio_width: Width component of sensor aspect ratio (default 4)
        aspect_ratio_height: Height component of sensor aspect ratio (default 3)

    Returns:
        (sensor_width_mm, sensor_height_mm, sensor_diagonal_mm)
    """
    if not focal_length_mm or not focal_length_35mm:
        return None, None, None

    # Compute crop factor from focal lengths
    crop_factor = focal_length_35mm / focal_length_mm

    print(f"Crop Factor: {crop_factor}")

    # Full-frame diagonal (36x24 mm)
    full_frame_width = 36.0
    full_frame_height = 24.0
    full_frame_diagonal = math.sqrt(full_frame_width**2 + full_frame_height**2)

    # Sensor diagonal using crop factor (based on FOV equivalence)
    sensor_diagonal = full_frame_diagonal / crop_factor

    # Compute width and height from sensor diagonal and aspect ratio
    ratio_diagonal = math.sqrt(aspect_ratio_width**2 + aspect_ratio_height**2)
    sensor_width = sensor_diagonal * (aspect_ratio_width / ratio_diagonal)
    sensor_height = sensor_diagonal * (aspect_ratio_height / ratio_diagonal)

    return sensor_width, sensor_height, sensor_diagonal

aspect_ratio_w = 4
aspect_ratio_h = 3

# Estimate sensor dimensions
sensor_width_mm, sensor_height_mm, sensor_diagonal_mm = estimate_sensor_dimensions(
    focal_length, focal_length_35mm, aspect_ratio_w, aspect_ratio_h
)

print(f"Estimated sensor width: {sensor_width_mm:.2f} mm")
print(f"Estimated sensor height: {sensor_height_mm:.2f} mm")
print(f"Estimated sensor diagonal: {sensor_diagonal_mm:.2f} mm")


Crop Factor: 4.362416079460911
Estimated sensor width: 7.93 mm
Estimated sensor height: 5.95 mm
Estimated sensor diagonal: 9.92 mm


In [11]:
def calculate_object_size(pixel_length, image_width_pixels, sensor_width_mm, focal_length_mm, distance_mm):
    """
    Calculate real-world object size from pixel measurements.
    
    Formula:
    1. pixel_size_mm = sensor_width_mm / image_width_pixels
    2. object_size_on_sensor_mm = pixel_length × pixel_size_mm
    3. real_object_size_mm = (object_size_on_sensor_mm × distance_mm) / focal_length_mm
    
    Args:
        pixel_length: Length of object in pixels
        image_width_pixels: Image width in pixels
        sensor_width_mm: Camera sensor width in mm
        focal_length_mm: Focal length in mm
        distance_mm: Distance from camera to object in mm
    
    Returns:
        Real object size in mm
    """
    # Calculate pixel size in mm
    pixel_size_mm = sensor_width_mm / image_width_pixels
    
    # Calculate object size on sensor
    object_size_on_sensor_mm = pixel_length * pixel_size_mm
    
    # Calculate real object size using similar triangles
    real_object_size_mm = (object_size_on_sensor_mm * distance_mm) / focal_length_mm
    
    return real_object_size_mm

# Your data
image_width_pixels = image_width
focal_length_mm = focal_length
sensor_width_mm = sensor_width_mm
pixel_length = 2241.25

# You need to know the distance to the object
distance_mm = 600  # Example: 1000mm = 1 meter (you need to measure this!)

object_size_mm = calculate_object_size(
    pixel_length=pixel_length,
    image_width_pixels=image_width_pixels,
    sensor_width_mm=sensor_width_mm,
    focal_length_mm=focal_length_mm,
    distance_mm=distance_mm
)

print(f"Real object size: {object_size_mm:.2f} mm ({object_size_mm/10:.2f} cm)")

Real object size: 313.42 mm (31.34 cm)
